#Optimal Planning using the Unified Planning framework

This python notebook shows how to use the unified planning library to solve problems with a given optimality metric.

## Setup the library

We install (from github) the unified planning library.

In [ ]:
# begin of installation

In [ ]:
!pip install --pre unified-planning[fast-downward]

We are now ready to use the Unified-Planning library!

In [ ]:
# end of installation

## Demo


In this demo we show how to model a problem with an optimality metric and how to solve it following the given metric.

We start importing the shortcuts.

In [ ]:
from unified_planning.shortcuts import *

Now we start to model a basic problem with action costs.


### Creating the problem

#### Classical part

First, we define the `UserTypes` and the `Fluents`.

In [ ]:
# basic with actions cost
x = Fluent("x")
y = Fluent("y")
a = InstantaneousAction("a")
a.add_precondition(Not(x))
a.add_effect(x, True)
b = InstantaneousAction("b")
b.add_precondition(Not(y))
b.add_effect(y, True)
c = InstantaneousAction("c")
c.add_precondition(y)
c.add_effect(x, True)
problem = Problem("basic_with_costs")
problem.add_fluent(x)
problem.add_fluent(y)
problem.add_action(a)
problem.add_action(b)
problem.add_action(c)
problem.set_initial_value(x, False)
problem.set_initial_value(y, False)
problem.add_goal(x)
problem.add_quality_metric(
    up.model.metrics.MinimizeActionCosts({a: Int(10), b: Int(1), c: Int(1)})
)
expected_plan = up.plans.SequentialPlan(
    [up.plans.ActionInstance(b), up.plans.ActionInstance(c)]
)

We get a solver that guarantees that the problem cna be solved optimally.

In [ ]:
with OneshotPlanner(
    problem_kind=problem.kind,
    optimality_guarantee=PlanGenerationResultStatus.SOLVED_OPTIMALLY,
) as planner:
    self.assertNotEqual(planner, None)
    final_report = planner.solve(problem)
    plan = final_report.plan
    self.assertEqual(
        final_report.status, PlanGenerationResultStatus.SOLVED_OPTIMALLY
    )
    self.assertEqual(plan, expected_plan)

Here, we assume that the plan is the one we expected, and not the shortes one.

In [ ]:
this is defined in the 

#### Temporal part

Now we start with the temporal aspects creating two durative actions.

A durative action has a duration, a set of conditions associated to an interval/timing and a set of effects associated to a timing.

We define the `light_match` action setting a fixed duration and defining a condition at its start and three effects at its end.

In [ ]:
light_match = DurativeAction('light_match', m=Match)
m = light_match.parameter('m')
light_match.set_fixed_duration(6)
light_match.add_condition(StartTiming(), Not(match_used(m)))
light_match.add_effect(StartTiming(), match_used(m), True)
light_match.add_effect(StartTiming(), light, True)
light_match.add_effect(EndTiming(), light, False)
problem.add_action(light_match)
print(light_match)

Defining the `mend_fuse` action we defined also a condition over an interval.

In [ ]:
mend_fuse = DurativeAction('mend_fuse', f=Fuse)
f = mend_fuse.parameter('f')
mend_fuse.set_fixed_duration(5)
mend_fuse.add_condition(StartTiming(), handfree)
mend_fuse.add_condition(ClosedTimeInterval(StartTiming(), EndTiming()), light)
mend_fuse.add_effect(StartTiming(), handfree, False)
mend_fuse.add_effect(EndTiming(), fuse_mended(f), True)
mend_fuse.add_effect(EndTiming(), handfree, True)
problem.add_action(mend_fuse)
print(mend_fuse)

We conclude the modeling defining three goals at the end of the execution.

In [ ]:
for f in fuses:
  problem.add_timed_goal(EndTiming(), fuse_mended(f))

print(problem)

### Solving the problem

The unified_planning can automatically select, among the available planners installed on the system, one that is able to handle the temporal features of the problem.

In [ ]:
with OneshotPlanner(problem_kind=problem.kind) as planner:
    result = planner.solve(problem)
    plan = result.plan
    if plan is not None:
        print("%s returned:" % planner.name)
        for start, action, duration in plan.timed_actions:
            print("%s: %s [%s]" % (float(start), action, float(duration)))
    else:
        print("No plan found.")